In [1]:
import cv2
import os
import numpy as np

from utils.metrics import compute_confusion_matrix

In [2]:
threshold = 0.5
mask_shape = (640, 640)
models = ['yolov8', 'segformer', 'mask2former']
pred_mask_folder = 'C:/Users/Anton Volodin/PycharmProjects/segmentation_ensemble/pred/'
gt_mask_folder = 'E:/datasets/task/dataset/gt_sem_seg/'

In [3]:
gt_mask_paths = [gt_mask_folder + filename for filename in os.listdir(gt_mask_folder)]
model_metrics = {model: {'iou': 0, 'precision': 0, 'recall': 0} for model in models}
pred_mask_paths = {}

In [4]:
for model in models:
    FP, FN, TP, TN = 0, 0, 0, 0
    pred_mask_paths[model] = [f'{pred_mask_folder}{model}/{filename}' for filename in os.listdir(pred_mask_folder + model)]
    for gt_mask_path, pred_mask_path in zip(gt_mask_paths, pred_mask_paths[model]):
        pred_mask = np.load(pred_mask_path)
        pred_mask = np.array(pred_mask > threshold, dtype=np.uint8)
        gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = np.array(gt_mask / 255, dtype=np.uint8)
        res = compute_confusion_matrix(gt_mask, pred_mask)
        FP += res[0]
        FN += res[1]
        TP += res[2]
        TN += res[3]
    print(f'Results for model: {model}')
    print(f'IoU is: {TP / (TP + FP + FN):3f}')
    print(f'Precision is: {TP / (TP + FP):3f}')
    print(f'Recall is: {TP / (TP + FN):3f}')

Results for model: yolov8
IoU is: 0.761480
Precision is: 0.938222
Recall is: 0.801676
Results for model: segformer
IoU is: 0.718954
Precision is: 0.727514
Recall is: 0.983898
Results for model: mask2former
IoU is: 0.652780
Precision is: 0.755061
Recall is: 0.828148


In [7]:
coefs = [[0.333, 0.333, 0.333],
         [0.5, 0.25, 0.25],
         [0.5, 0.3, 0.2]]
for coef in coefs:
    FP, FN, TP, TN = 0, 0, 0, 0
    for gt_mask_path in gt_mask_paths:
        filename = gt_mask_path.split('/')[-1].split('.')[0]
        combined_mask = np.zeros(mask_shape, dtype=np.float32)
        for i, model in enumerate(models):
            pred_mask_path = f'{pred_mask_folder}{model}/{filename}.npy'
            pred_mask = np.load(pred_mask_path)
            combined_mask += pred_mask * coef[i]
        combined_mask = np.array(combined_mask > threshold, dtype=np.uint8)
        gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = np.array(gt_mask / 255, dtype=np.uint8)
        res = compute_confusion_matrix(gt_mask, combined_mask)
        FP += res[0]
        FN += res[1]
        TP += res[2]
        TN += res[3]
    print(f'Combined model coefs: ', coef)
    print(f'Combined model iou is: {TP / (TP + FP + FN):.3f}')
    print(f'Combined model  precision is: {TP / (TP + FP):.3f}')
    print(f'Combined model  recall is: {TP / (TP + FN):.3f}')
    print('-'* 50)

Combined model coefs:  [0.333, 0.333, 0.333]
Combined model iou is: 0.820
Combined model  precision is: 0.919
Combined model  recall is: 0.884
--------------------------------------------------
Combined model coefs:  [0.5, 0.25, 0.25]
Combined model iou is: 0.799
Combined model  precision is: 0.950
Combined model  recall is: 0.835
--------------------------------------------------
Combined model coefs:  [0.5, 0.3, 0.2]
Combined model iou is: 0.804
Combined model  precision is: 0.950
Combined model  recall is: 0.839
--------------------------------------------------
